In [ ]:
#default_exp common.metrics

In [ ]:
#export
from fastai2.basics import *
from fastai2.callback.all import *
from fastcook.utils import *

# Metrics
> Recipes on how to create and use custom metrics.

## Probably what you want

If your metric consists of computing a value using the predictions and the labels, and then averaging over all points, use `AvgMetric`.  
You only need to define a function that receives two arguments, the prediction and the targets, and returns a single scalar.

In [ ]:
def crazy_metric(pred,targ): return (pred>targ).float().mean()
CrazyMetric = AvgMetric(crazy_metric)

In [ ]:
learn = synth_learner(metrics=CrazyMetric)
learn.fit(2)

epoch,train_loss,valid_loss,crazy_metric,time
0,24.437227,21.049934,0.031250,00:00
1,24.459761,20.942612,0.031250,00:00


## More control

To have full control on all steps of calculating a metric inherit from `Metric`, the three methods needed to override are `reset`, `accumulate` and `value`.  
* `reset` is called at the beggining of the validation step, here you should initialize all the required variables.
* `accumulate` is called after every batch, here you do the actual calculation of your metric and decide how to accumulate the values between the batches.
* `value` is called at the end of the validation step, it should return the final value of your already calculated metric.

In [ ]:
class EvenCrazierMetric(Metric):
    def reset(self): self.count,self.total = 0,0
    def accumulate(self, learn):
        bs = find_bs(learn.yb)
        pred,yb = learn.pred, detuplify(learn.yb)
        self.count = self.count*0.2 + 0.8*(pred-yb).float().sum()
        self.total += bs
    @property
    def value(self): return self.count*self.total

In [ ]:
learn = synth_learner(metrics=EvenCrazierMetric())
learn.fit(2)

epoch,train_loss,valid_loss,even_crazier,time
0,24.219156,23.979538,-2295.573242,00:00
1,24.121483,23.860945,-2290.544434,00:00
